In [4]:
from datasets import load_dataset
from huggingface_hub import login
import os

ds1 = load_dataset("aripos1/gorani_dataset")

hf_token = os.getenv('HUGGING_FACE_TOKEN')  # 보안 강화를 위해 입력받음
login(token=hf_token)  # Hugging Face 로그인

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")  
EOS_TOKEN = tokenizer.eos_token

# 2. Alpaca 스타일 프롬프트 템플릿 정의
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Glossary:
{}

## target_language:
{}

### Response:
{}"""  

def apply_prompt_template(row):
    glossary_text = row["metadata"] if isinstance(row["metadata"], str) else "N/A"
    target_language = row["target_language"] if isinstance(row["target_language"], str) else "N/A"
    return alpaca_prompt.format(row["instruction"], row["input"], glossary_text, target_language, row["output"], ) + EOS_TOKEN

# Modify the dataset using the apply_prompt_template function
ds1["train"] = ds1["train"].map(lambda x: {"text": apply_prompt_template(x)}, batched=False)
ds1["test"] = ds1["test"].map(lambda x: {"text": apply_prompt_template(x)}, batched=False)


Map:   0%|          | 0/21662 [00:00<?, ? examples/s]

Map:   0%|          | 0/5416 [00:00<?, ? examples/s]

In [7]:
ds1.push_to_hub("haeun0420/gorani_full_finetuning_datasets")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/haeun0420/gorani_full_finetuning_datasets/commit/4f085208b0b7cc9615b9385d2d37a6de90823d6c', commit_message='Upload dataset', commit_description='', oid='4f085208b0b7cc9615b9385d2d37a6de90823d6c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/haeun0420/gorani_full_finetuning_datasets', endpoint='https://huggingface.co', repo_type='dataset', repo_id='haeun0420/gorani_full_finetuning_datasets'), pr_revision=None, pr_num=None)